In [1]:
from dbfread import DBF
import pandas as pd
import string
from datetime import datetime
from pydantic import BaseModel
import json


In [2]:
TRASPASOS = 'Bases 2022/31-12-2022/Spm2003/2022/traspasos.DBF'
TRASPASOS21 = 'Bases 2021/31-12-2021/Spm2003/2021/traspasos.DBF'
COSTOS22 = 'Bases 2022/31-12-2022/Spm2003/2022/costos22.DBF'


In [3]:
def cargar_dbf_por_chunks(nombre_archivo, chunk_size=1000):
    """
    Carga un archivo DBF en memoria por chunks.

    Args:
    - nombre_archivo (str): El nombre del archivo DBF.
    - chunk_size (int): Tamaño del chunk para dividir los datos.

    Returns:
    - Generador de DataFrames: Un generador que produce DataFrames de tamaño chunk_size.
    """
    # Abre el archivo DBF
    dbf = DBF(nombre_archivo)

    # Inicializa un DataFrame vacío
    df = pd.DataFrame()

    # Itera sobre los registros y crea DataFrames por chunks
    for record in dbf:
        df = pd.concat([df, pd.DataFrame([record])], ignore_index=True)
        if len(df) >= chunk_size:
            # Produce el chunk actual como DataFrame
            yield df
            # Vacía el DataFrame
            df = pd.DataFrame()

    # Produce el último chunk si hay datos restantes
    if not df.empty:
        yield df

In [4]:
dbf_traspados = DBF(TRASPASOS)
dbf_traspasos2021 = DBF(TRASPASOS21)
dbf_costos22 = DBF(COSTOS22)

In [5]:
df_traspasos2021 = pd.DataFrame(iter(dbf_traspasos2021))
df_traspasos2022 = pd.DataFrame(iter(dbf_traspados))
df_traspasos = pd.concat([df_traspasos2021, df_traspasos2022], ignore_index=True)



In [6]:
df_traspasos.head()

,NROTRASP,ORDEN,DCCOSTO,ACCOSTO,CANT_PR,FECHORA,ESTADO,USUARIO,OBS_TRP
0,522710,,680,115,NaN,2021-01-07 09:53:41.999,,JJPEREZ,None
1,522711,,680,115,NaN,2021-01-07 09:58:10.000,,JJPEREZ,None
2,522712,,115,118,NaN,2021-01-07 10:00:13.999,,JJPEREZ,None
3,522713,,670,680,NaN,2021-01-07 10:03:38.000,,JJPEREZ,None
4,522714,,680,115,NaN,2021-01-07 10:05:28.000,,JJPEREZ,None


In [7]:
def binary_search(df, target_value):
    left, right = 0, len(df) - 1
    while left <= right:
        mid = (left + right) // 2
        mid_value = df.iloc[mid]['NROTRASP']
        if mid_value == target_value:
            return df.iloc[mid]  # Se encontró el valor, devuelve toda la fila
        elif mid_value < target_value:
            left = mid + 1
        else:
            right = mid - 1
    return None 

In [8]:
item = binary_search(df_traspasos, 90000)
print(item)

None


In [36]:
class TraspasoDBF(BaseModel):
    NROTRASP: int 
    DCCOSTO: str
    ACCOSTO: str
    FECHORA: datetime | None
    TIME: float | None

In [37]:
class PrendaDBF(BaseModel):
      PEDIDO : str
      ORDEN : str
      CODCLI : str
      CLIENTE : str
      CANTIDAD : int
      INICIO : datetime | None
      ENTREGA : datetime | None
      CODART : str
      ARTICULO  : str
      MODELO : str
      NMODELO: str
      FORRO : str
      CUELLO : str
      ABCERR: str
      COLOR : str
      NCOLOR: str
      TALLA: str
      MEDIDA: str
      NMEDIDA: str
      TIPOLANA: str
      TITULO: str
      NTITULO: str
      CBARRAS: str
      ECBARRAS: str
      FECHA: datetime | None
      CCACTUAL: str 
      NCCACTUAL: str 
      PASO: int | None 
      VEZ: int | None
      TALLER: str
      NTALLER: str
      PRENDA: str
      NPRENDA: str
      CCALIDAD: int | None
      CODIGO: str
      # TRASPASOS : list[TraspasoDBF] | None

In [42]:
chunksCostos = cargar_dbf_por_chunks(COSTOS22, 10)

for chunk in chunksCostos:
    # write json chunk.columns
    for index, row in chunk.iterrows():
        obj = PrendaDBF(**row)
        # print(obj.model_dump())
        traspasos = []
        last_fechhora = None
        for i in range(1, 25):
            paso = row[f"PASO{i}"]
            if paso > 0:
                objTraspaso = {}
                foundTraspaso = binary_search(df_traspasos, paso)
                if last_fechhora is not None and foundTraspaso["FECHORA"] is not None:
                    current_fechora = foundTraspaso["FECHORA"]
                    objTraspaso["TIME"]=  (current_fechora - last_fechhora).total_seconds()
                else :
                    objTraspaso["TIME"] = None
                objTraspaso["TRAPASO"] = foundTraspaso.to_dict()
                objTraspaso["INDEX"] = i
                objTraspaso["PASO"] = paso
                objTraspaso["FECHA"] = row[f"FECHA{i}"]
                objTraspaso["VEZ"] = row[f"VEZ{i}"]
                traspasos.append(objTraspaso)
                if(foundTraspaso["FECHORA"] is not None):
                    last_fechhora = foundTraspaso["FECHORA"]

        objNew = obj.model_dump()
        objNew["TRASPASOS"] = traspasos
        with(open(f"{obj.CODART}.json", "w")) as file:
            json.dump(objNew, file, indent=4, default=str)
        break
    break